---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [9]:
def degree_distribution(G):
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes( G)) for i in degree_values]
    return histogram

def graph_identification():
    methods = []
    for G in P1_Graphs:
        clustering = nx.average_clustering(G)
        shortest_path = nx.average_shortest_path_length(G)
        degree_hist = degree_distribution(G)
        if len(degree_hist)>10:
            methods.append('PA')
        elif clustering < 0.1:
            methods.append('SW_H')
        else:
            methods.append('SW_L')
    # Your Code Here
    
    return methods# Your Answer Here

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [10]:
def salary_predictions():
   
    #nx.to_pandas_dataframe(graph_name) use to convert a graph object to a data frame object
    #nx.from_pandas_dataframe(graph_name) use to convert  a data frame object to a graph object
    lst=[]
    for i in nx.get_node_attributes(G,'ManagementSalary').values():
        lst.append(i)
    
    
    df = pd .DataFrame(index=G.nodes())
    df['management_salary']=lst

    lst1=[]
    lst2=[]
    for i in G.nodes():
        lst1.append(nx.degree(G,i))
        lst2.append(nx.clustering(G,i))

    df['clustering']=lst2
    df['degree']=lst1

    df['degree_centrality']=[i for i in nx.degree_centrality(G).values()]
    df['closeness_centrality']=[i for i in nx.closeness_centrality(G).values()]
    df['betweenness_centrality']=[i for i in nx.betweenness_centrality(G).values()]
    df['pagerank']=[i for i in nx.pagerank(G).values()]



    train_data = df[(df['management_salary']==1) | (df['management_salary']==0)]

    test_data = df[np.isnan(df['management_salary'])]

    test_data.dropna(axis=1,inplace=True)

##############################################################################

    test_data.dropna(axis=1,inplace=True)


##############################################################################

    X = train_data[['clustering','degree','degree_centrality','closeness_centrality','betweenness_centrality','pagerank']]
    y=train_data['management_salary']
    
    from sklearn.neural_network import MLPClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler
        
    X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=True)

    scaler = MinMaxScaler()

    X_tarin_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    scaled_test_data = scaler.transform(test_data)

    neucls = MLPClassifier(hidden_layer_sizes=(10,5),solver='lbfgs',alpha=0.01 ,verbose=0).fit(X_tarin_scaled,y_train)
# these are to check the model is working good
    pred = neucls.predict(X_test_scaled)

    sco = neucls.score(X_test_scaled,y_test)

    from sklearn.metrics import roc_curve , auc , roc_auc_score

    sc= roc_auc_score(pred,y_test)

#####################################################

    pred1 = neucls.predict(scaled_test_data)

    predic = neucls.predict_proba(scaled_test_data)

    test_data['predi']=predic[:,1]
    last_ser=pd.Series(test_data['predi'],test_data.index)
    
    # Your Code Here
    
    return last_ser# Your Answer Here

salary_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1       3.914486e-03
2       9.056898e-01
5       1.000000e+00
8       1.579537e-01
14      2.049977e-02
18      1.570869e-01
27      4.153782e-01
30      3.798156e-01
31      6.186823e-02
34      3.222247e-02
37      3.393614e-03
40      3.601295e-02
45      1.504121e-01
54      5.021742e-01
55      9.856445e-01
60      3.253273e-02
62      9.999993e-01
65      9.955658e-01
77      1.737093e-02
79      2.127795e-02
97      2.123541e-02
101     1.900812e-03
103     2.465687e-01
108     2.654048e-02
113     2.272749e-01
122     9.711727e-05
141     3.703006e-01
142     5.693792e-01
144     9.799959e-03
145     8.449089e-01
            ...     
913     3.123453e-04
914     2.266031e-03
915     1.154818e-12
918     8.743304e-02
923     4.924279e-06
926     8.379488e-03
931     7.799456e-04
934     4.639824e-07
939     5.158821e-08
944     4.543314e-12
945     2.759118e-05
947     7.871478e-02
950     1.080215e-03
951     1.416272e-05
953     2.319516e-05
959     5.425190e-11
962     5.004

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [4]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [6]:
def new_connections_predictions():
    import numpy as np
    G = nx.read_gpickle('email_prediction.txt')
    G.nodes(data=True)

# here we are adding a node attribute community because soundarajan_hopcroft need community info to calculate
# here we are using department attribute of nodes as a community
    for i in G.nodes():
        G.node[i]['community']=G.node[i]['Department']


#for preferantial attachment

    pre = nx.preferential_attachment(G) 

    lst=[]
    for i in pre:
        lst.append(i)
    
    df_pre=pd.DataFrame(index=([(i[0],i[1]) for i in lst]))

    df_pre['pefer_attch']=[i[2] for i in lst]

    last_df=pd.merge(future_connections,df_pre,how='inner',left_index=True,right_index=True)

# for jaccard coefficient

    jac = nx.jaccard_coefficient(G) 

    lst1=[]
    for i in jac:
        lst1.append(i)
    
    df_jac=pd.DataFrame(index=([(i[0],i[1]) for i in lst1]))

    df_jac['jacc_coef']=[i[2] for i in lst1]
    df_jac

    last_df=pd.merge(last_df,df_jac,how='inner',left_index=True,right_index=True)

# for soundarajan_hopcroft score

    sou = nx.cn_soundarajan_hopcroft(G) 

    lst2=[]
    for i in sou:
        lst2.append(i)
    
    df_sou=pd.DataFrame(index=([(i[0],i[1]) for i in lst2]))

    df_sou['sou_hop']=[i[2] for i in lst2]
    df_sou

    last_df=pd.merge(last_df,df_sou,how='inner',left_index=True,right_index=True)

# for resource allocation index

    reso = nx.resource_allocation_index(G) 

    lst3=[]
    for i in reso:
        lst3.append(i)
    
    df_reso=pd.DataFrame(index=([(i[0],i[1]) for i in lst3]))

    df_reso['reso_ind']=[i[2] for i in lst3]
    df_reso

    last_df=pd.merge(last_df,df_reso,how='inner',left_index=True,right_index=True)

# the data frame is created

    train_data = last_df[~np.isnan(last_df['Future Connection'])] 
# the '~' symbol is use to show that which are not satisfying the condition
    test_data = last_df[np.isnan(last_df['Future Connection'])]

    test_data.dropna(axis=1 , inplace=True) # this one is to drop the Future Connection column

    X=train_data[['pefer_attch','jacc_coef','sou_hop','reso_ind']]
    y=train_data['Future Connection']

    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.neural_network import MLPClassifier
    from sklearn.metrics import roc_auc_score

    X_train , X_test , y_train , y_test = train_test_split(X,y,random_state=0)


    scaler = MinMaxScaler()
    X_train_scl = scaler.fit_transform(X_train)
    X_test_scl = scaler.transform(X_test)
    test_data_scaled = scaler.transform(test_data)


    neu = MLPClassifier(hidden_layer_sizes=(10,5),solver='lbfgs').fit(X_train_scl,y_train)


# this one is just to check the model efficienency
    prediction = neu.predict(X_test_scl)
    predic = neu.predict(test_data_scaled)

    roasc= roc_auc_score(prediction,y_test)


# now we have to predict the test data


    pred_prob = neu.predict_proba(test_data_scaled)


    test_data['predi']=pred_prob[:,1]

    last_ser=pd.Series(test_data['predi'],test_data.index)
    
    # Your Code Here
    
    return last_ser # Your Answer Here


new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.038306
(542, 751)    0.012726
(20, 426)     0.585280
(50, 989)     0.012955
(942, 986)    0.013204
(324, 857)    0.012923
(13, 710)     0.172523
(19, 271)     0.104715
(319, 878)    0.013035
(659, 707)    0.012749
(49, 843)     0.013107
(208, 893)    0.012804
(377, 469)    0.005859
(405, 999)    0.024583
(129, 740)    0.019604
(292, 618)    0.027049
(239, 689)    0.013051
(359, 373)    0.008172
(53, 523)     0.102209
(276, 984)    0.013095
(202, 997)    0.013143
(604, 619)    0.035913
(270, 911)    0.013047
(261, 481)    0.065138
(200, 450)    0.994941
(213, 634)    0.012656
(644, 735)    0.031125
(346, 553)    0.012100
(521, 738)    0.011081
(422, 953)    0.019195
                ...   
(672, 848)    0.013047
(28, 127)     0.960200
(202, 661)    0.012008
(54, 195)     0.999997
(295, 864)    0.012887
(814, 936)    0.012691
(839, 874)    0.013204
(139, 843)    0.012907
(461, 544)    0.010407
(68, 487)     0.010292
(622, 932)    0.012832
(504, 936)    0.015832
(479, 528) 